In [1]:
from Util import prepare_Task, TaskID_Generator
import os 
from Config import DataSetConfig, ModelConfig
import ClassificationTask_Level 
import DataSet
import numpy as np

Classes = []
configs = []

Name = "Cardiomegaly"
Task = "Segmentation"
DataSetPath = "Generated_Data_20180410_191400_Seg_Base/Imgs"

Classes.append("Tx")

GPU_Num, BaseDataPath = prepare_Task(Task)
dataConfig = DataSetConfig(Name, (256,256,1), Classes, DataSetPath)
os.environ["CUDA_VISIBLE_DEVICES"] = GPU_Num
#configs.append(config)

train_data, train_files, train_labels, mean,std = DataSet.get_Data_Set_Level(dataConfig, BaseDataPath,"train")
valid_data, valid_files, valid_labels, _,_ = DataSet.get_Data_Set_Level(dataConfig, BaseDataPath,"validation")
test_data, valid_files, test_labels, _ , _= DataSet.get_Data_Set_Level(dataConfig, BaseDataPath,"test")
train_data = np.asarray(train_data)
train_data -= mean
valid_data -= mean
test_data -= mean

train_data /= std
valid_data /= std
test_data /= std


/home/gyungchan2110/gyungchan2110_python3/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


/data/gyungchan2110/Segmentation/Generated_Data_20180410_191400_Seg_Base/Imgs/train
train 1209
 : --- 68.88090586662292 seconds ---
/data/gyungchan2110/Segmentation/Generated_Data_20180410_191400_Seg_Base/Imgs/validation
validation 401
 : --- 23.148021936416626 seconds ---
/data/gyungchan2110/Segmentation/Generated_Data_20180410_191400_Seg_Base/Imgs/test
test 410
 : --- 23.17786931991577 seconds ---


In [2]:
import tensorflow as tf 
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

ModelConfigs = []

# config = ModelConfig(Name, "XCEPTION", "ADAM",16, 1e-5)
# config.HEIGHT_SHIFT_RANGE = 0.25
# config.WIDTH_SHIFT_RANGE = 0.1
# config.ROTATION_RANGE = 0
# config.ZOOM_RANGE = 0.06
# ModelConfigs.append(config)

config = ModelConfig(Name, "CNN6Layers", "SGD",8, 1e-4)
config.HEIGHT_SHIFT_RANGE = 0.25
config.WIDTH_SHIFT_RANGE = 0.1
config.ROTATION_RANGE = 0
config.ZOOM_RANGE = 0.06
config.LOSS = "mean_squared_error"
ModelConfigs.append(config)

#ModelConfigs.append(config)


# for i in range(10):
#     for j in range(10):
#         for k in range(10):
#             for m in range(10):
#                 config = ModelConfig(Name, "XCEPTION", "ADAM",16, 1e-4)
#                 config.ROTATION_RANGE = i * 2
#                 config.WIDTH_SHIFT_RANGE = 0.05*j
#                 config.HEIGHT_SHIFT_RANGE = 0.05*k
#                 config.ZOOM_RANGE = 0.01*m

#                 ModelConfigs.append(config)
    
# Train_TaskIDs = []


for i in range(len(ModelConfigs)):
    GPU_Num, BaseDataPath = prepare_Task("Classification")
    TaskID = TaskID_Generator() 
    os.environ["CUDA_VISIBLE_DEVICES"] = GPU_Num
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
    print(GPU_Num)
    result , filename = ClassificationTask_Level.Train_DataSet(TaskID, BaseDataPath, dataConfig, ModelConfigs[i], train_data, valid_data, train_labels, valid_labels)
    trainID = "Train_" + TaskID
    Train_TaskIDs.append(trainID)
    if(result):
        ClassificationTask_Level.Test_Dataset(TaskID, BaseDataPath, dataConfig, filename, test_data, test_labels)

3
Train_20180509_114727
152 51
Error when checking input: expected input_1 to have 4 dimensions, but got array with shape (256, 256, 1)
Train Done ! 


NameError: name 'Train_TaskIDs' is not defined

In [ ]:
for Train_TaskID in Train_TaskIDs: 
    ModelsFolder = BaseDataPath + "/LOGS/logs/" + Train_TaskID

    filenames = os.listdir(ModelsFolder)
    count = 0
    print(filenames)
    for filename in filenames:
        full_filename = os.path.join(ModelsFolder, filename)
        ext = os.path.splitext(full_filename)[-1]
        if ext == '.hdf5': 
            ModelFile = Train_TaskID + "/" + filename
            GPU_Num, BaseDataPath = prepare_Task("Classification")
            TaskID = TaskID_Generator() 
            os.environ["CUDA_VISIBLE_DEVICES"] = GPU_Num
            print(filename)
            ClassificationTask.Test_Dataset(TaskID, BaseDataPath, dataConfig, ModelFile, test_data, test_labels, True)
